In [0]:
# Set the storage account key
spark.conf.set(
    
    
)

# List the files in the specified directory
display(dbutils.fs.ls("abfss://raw-data@nyctaxiproject003.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://raw-data@nyctaxiproject003.dfs.core.windows.net/yellow_tripdata_2020-01.csv,yellow_tripdata_2020-01.csv,6335,1753244849000


In [0]:
spark.conf.set(
    
)
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://raw-data@nyctaxiproject003.dfs.core.windows.net/yellow_tripdata_2020-01-01.csv")


In [0]:
df = spark.read.format("csv") \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .load("abfss://raw-data@nyctaxiproject003.dfs.core.windows.net/yellow_tripdata_2020-01.csv")

df.show(10)


+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1|Wed Jan 01 00:28:...| Wed Jan 01 00:33:...|            1.0|          1.2|       1.0|                 N|         238|         239|           1|        6.0|  3.0|    0.5|      1.4

In [0]:
from pyspark.sql.functions import col

df = df.withColumn(
    "Revenue",
    col("fare_amount") +
    col("extra") +
    col("mta_tax") +
    col("improvement_surcharge") +
    col("tip_amount") +
    col("tolls_amount") +
    col("total_amount")
)

# Show a sample of the new Revenue column
df.select("fare_amount", "tip_amount", "total_amount", "Revenue").show(10)


+-----------+----------+------------+-----------------+
|fare_amount|tip_amount|total_amount|          Revenue|
+-----------+----------+------------+-----------------+
|        6.0|      1.47|       11.27|            22.54|
|        7.0|       1.5|        12.3|             24.6|
|        6.0|       1.0|        10.8|             21.6|
|        5.5|      1.36|        8.16|            16.32|
|        3.5|       0.0|         4.8|              9.6|
|        2.5|       0.0|         3.8|              7.6|
|        2.5|      0.01|        3.81|7.619999999999999|
|       0.01|       0.0|        2.81|             3.12|
|        2.5|       0.0|         6.3|             10.1|
|        8.0|      2.35|       14.15|             28.3|
+-----------+----------+------------+-----------------+
only showing top 10 rows


In [0]:
# Group by Pickup location ID and sum the number of passengers
from pyspark.sql.functions import sum

passenger_by_area = df.groupBy("PULocationID") \
    .agg(sum("passenger_count").alias("Total_Passengers")) \
    .orderBy("Total_Passengers", ascending=False)  # Sort biggest first

# Show the top 10 areas by passengers
passenger_by_area.show(10)


+------------+----------------+
|PULocationID|Total_Passengers|
+------------+----------------+
|         193|             7.0|
|         246|             7.0|
|          95|             6.0|
|         239|             4.0|
|         234|             3.0|
|         142|             3.0|
|         211|             3.0|
|         144|             3.0|
|         141|             3.0|
|         236|             3.0|
+------------+----------------+
only showing top 10 rows


In [0]:
from pyspark.sql.functions import avg

# Group rides by Vendor ID and calculate average fare and total earnings
avg_vendor = df.groupBy("VendorID") \
    .agg(
        avg("fare_amount").alias("Average_Fare"),
        avg("total_amount").alias("Average_Total_Earnings")
    ) \
    .orderBy("VendorID")

# Show average fares and total earnings for each vendor
avg_vendor.show()


+--------+------------------+----------------------+
|VendorID|      Average_Fare|Average_Total_Earnings|
+--------+------------------+----------------------+
|       1|               9.0|     14.50088235294118|
|       2|10.588823529411764|    14.996470588235297|
+--------+------------------+----------------------+



In [0]:
from pyspark.sql.functions import col

# Count number of rides (rows) for each payment type
payment_counts = df.groupBy("payment_type") \
    .count() \
    .orderBy(col("count").desc())

# Show the results
payment_counts.show()


+------------+-----+
|payment_type|count|
+------------+-----+
|           1|   36|
|           2|   14|
|           4|    1|
+------------+-----+



In [0]:
#   query 05
from pyspark.sql.functions import to_date, sum as spark_sum

# First, convert pickup datetime into a new column showing only the date
df = df.withColumn("trip_date", to_date("tpep_pickup_datetime"))


In [0]:
# Now group by Vendor and Date to get total:
# - Earnings
# - Passengers
# - Distance
vendor_stats = df.groupBy("VendorID", "trip_date") \
    .agg(
        spark_sum("total_amount").alias("Total_Earnings"),
        spark_sum("passenger_count").alias("Total_Passengers"),
        spark_sum("trip_distance").alias("Total_Distance")
    )


In [0]:
# Now filter for the date "2020-01-15"
vendor_on_date = vendor_stats.filter(vendor_stats.trip_date == "2020-01-15")
top_2_vendors = vendor_on_date.orderBy("Total_Earnings", ascending=False).limit(2)
top_2_vendors.show()


+--------+---------+--------------+----------------+--------------+
|VendorID|trip_date|Total_Earnings|Total_Passengers|Total_Distance|
+--------+---------+--------------+----------------+--------------+
+--------+---------+--------------+----------------+--------------+



In [0]:
#query 06
from pyspark.sql.functions import sum as spark_sum

# Group by route (pickup + dropoff) and count all passengers
route_passengers = df.groupBy("PULocationID", "DOLocationID") \
    .agg(spark_sum("passenger_count").alias("Total_Passengers")) \
    .orderBy("Total_Passengers", ascending=False)

# Show the top 10 busiest routes
route_passengers.show(10)


+------------+------------+----------------+
|PULocationID|DOLocationID|Total_Passengers|
+------------+------------+----------------+
|         193|         193|             7.0|
|          95|          95|             6.0|
|         211|         234|             3.0|
|         246|          48|             3.0|
|         246|          79|             2.0|
|         141|         140|             2.0|
|         234|         238|             2.0|
|         148|          49|             2.0|
|         239|         142|             2.0|
|         236|         262|             2.0|
+------------+------------+----------------+
only showing top 10 rows


In [0]:
#query07
from pyspark.sql.functions import unix_timestamp, max as spark_max

# Convert pickup timestamp to epoch second time (a big number)
df = df.withColumn("pickup_epoch", unix_timestamp("tpep_pickup_datetime"))

# Get the maximum/latest pickup time in the dataset
max_time = df.select(spark_max("pickup_epoch")).collect()[0][0] if df.count() > 0 else None
print(f"Max timestamp in dataset: {max_time}")




Max timestamp in dataset: None


In [0]:
# Calculate the maximum time in the dataset
max_time = df.agg({"pickup_epoch": "max"}).collect()[0][0]

# Check if max_time is not None
if max_time is not None:
    # Only keep trips started in the last 10 seconds of the dataset
    recent_df = df.filter(df.pickup_epoch >= max_time - 10)
else:
    # Handle the case where max_time is None
    recent_df = spark.createDataFrame([], df.schema)

display(recent_df)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Revenue,trip_date,pickup_epoch


In [0]:
from pyspark.sql.functions import sum as spark_sum

# Sum up passengers from recent pickups
recent_pickups = recent_df.groupBy("PULocationID") \
    .agg(spark_sum("passenger_count").alias("Last_10sec_Passengers")) \
    .orderBy("Last_10sec_Passengers", ascending=False)

# Show the result
recent_pickups.show()


+------------+---------------------+
|PULocationID|Last_10sec_Passengers|
+------------+---------------------+
+------------+---------------------+



In [0]:
passenger_by_area.write.mode("overwrite") \
    .parquet("abfss://raw-data@nyctaxiproject003.dfs.core.windows.net/results/passenger_by_area.parquet")


In [0]:
passenger_by_area.coalesce(1).write.mode("overwrite") \
    .option("header", "true") \
    .csv("abfss://raw-data@nyctaxiproject003.dfs.core.windows.net/results/passenger_by_area_csv")
